# 파이토치

- 논문에 나와있는 kernel size, stride, padding을 따르면 나오는 feature map의 크기가 코드 실행 결과와 같은데, 그러면 파라미터 수가 논문보다 더 적어진다. 그런데 파라미터 수를 논문에 맞추려면 padding 같은 수치들을 바꿔줘야한다. 그러면 중간중간 아웃풋으로 나오는 feature map의 크기도 달라진다.

In [1]:
import torch
import torch.nn as nn


class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=False)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        x = self.conv(x)
        x = self.relu(x)
        
        return x
    

class Fire(nn.Module):
    def __init__(self, in_ch, s1x1):
        super(Fire, self).__init__()
        
        self.conv_s1x1 = ConvBlock(in_ch, s1x1, kernel_size=1, stride=1, padding=0)
        
        self.conv_e1x1 = ConvBlock(s1x1, s1x1*4, kernel_size=1, stride=1, padding=0)
        self.conv_e3x3 = ConvBlock(s1x1, s1x1*4, kernel_size=3, stride=1, padding=1)
        
    def forward(self, x):
        
        x = self.conv_s1x1(x)
        
        x_e1 = self.conv_e1x1(x)
        x_e3 = self.conv_e3x3(x)
        
        return torch.cat([x_e1, x_e3], dim=1)

In [2]:
class SqueezeNet_Vanilla(nn.Module):
    def __init__(self):
        super(SqueezeNet_Vanilla, self).__init__()
        
        self.conv1 = ConvBlock(3, 96, kernel_size=7, stride=2, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire_block1 = nn.Sequential(Fire(96, 16),    # fire2
                                         Fire(128, 16),   # fire3
                                         Fire(128, 32))   # fire4
        
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire_block2 = nn.Sequential(Fire(256, 32),   # fire5
                                         Fire(256, 48),   # fire6
                                         Fire(384, 48),   # fire7
                                         Fire(384, 64))   # fire8
        
        self.maxpool8 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire_block3 = nn.Sequential(Fire(512, 64))   # fire9
        
        self.dropout = nn.Dropout(0.5)
        
        self.conv10 = ConvBlock(512, 1000, kernel_size=1, stride=1, padding=0)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.fire_block1(x)
        x = self.maxpool4(x)
        x = self.fire_block2(x)
        x = self.maxpool8(x)
        x = self.fire_block3(x)
        x = self.dropout(x)
        x = self.conv10(x)
        x = self.avgpool(x)
        
        x = x.view(x.size(0),-1)
        
        return x

In [3]:
if __name__ == '__main__':

    from torchsummary import summary
    model = SqueezeNet_Vanilla()
    summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 111, 111]          14,112
              ReLU-2         [-1, 96, 111, 111]               0
         ConvBlock-3         [-1, 96, 111, 111]               0
         MaxPool2d-4           [-1, 96, 55, 55]               0
            Conv2d-5           [-1, 16, 55, 55]           1,536
              ReLU-6           [-1, 16, 55, 55]               0
         ConvBlock-7           [-1, 16, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]           1,024
              ReLU-9           [-1, 64, 55, 55]               0
        ConvBlock-10           [-1, 64, 55, 55]               0
           Conv2d-11           [-1, 64, 55, 55]           9,216
             ReLU-12           [-1, 64, 55, 55]               0
        ConvBlock-13           [-1, 64, 55, 55]               0
             Fire-14          [-1, 128,

In [4]:
class SqueezeNet_Simple_Bypass(nn.Module):
    def __init__(self):
        super(SqueezeNet_Simple_Bypass, self).__init__()
        
        self.conv1 = ConvBlock(3, 96, kernel_size=7, stride=2, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire2 = Fire(96, 16)
        self.fire3 = Fire(128, 16)
        self.fire4 = Fire(128, 32)
        
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire5 = Fire(256, 32)
        self.fire6 = Fire(256, 48)
        self.fire7 = Fire(384, 48)
        self.fire8 = Fire(384, 64)
        
        self.maxpool8 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire9 = Fire(512, 64)
        
        self.dropout = nn.Dropout(0.5)
        
        self.conv10 = ConvBlock(512, 1000, kernel_size=1, stride=1, padding=0)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        
        x = self.fire2(x)
        pre_xs = x # s: simple
        
        x = self.fire3(x)
        x = self.fire4(pre_xs+x)
        
        x = self.maxpool4(x)
        pre_xs = x
        
        x = self.fire5(x)
        x = self.fire6(pre_xs+x)
        pre_xs = x
        x = self.fire7(x)
        x = self.fire8(pre_xs+x)
        
        x = self.maxpool8(x)
        pre_xs = x
        x = self.fire9(x)
        
        x = self.dropout(x)
        
        x = self.conv10(pre_xs+x)
        x = self.avgpool(x)
        
        x = x.view(x.size(0),-1)
        
        return x

In [5]:
if __name__ == '__main__':

    from torchsummary import summary
    model = SqueezeNet_Simple_Bypass()
    summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 111, 111]          14,112
              ReLU-2         [-1, 96, 111, 111]               0
         ConvBlock-3         [-1, 96, 111, 111]               0
         MaxPool2d-4           [-1, 96, 55, 55]               0
            Conv2d-5           [-1, 16, 55, 55]           1,536
              ReLU-6           [-1, 16, 55, 55]               0
         ConvBlock-7           [-1, 16, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]           1,024
              ReLU-9           [-1, 64, 55, 55]               0
        ConvBlock-10           [-1, 64, 55, 55]               0
           Conv2d-11           [-1, 64, 55, 55]           9,216
             ReLU-12           [-1, 64, 55, 55]               0
        ConvBlock-13           [-1, 64, 55, 55]               0
             Fire-14          [-1, 128,

In [6]:
class SqueezeNet_Complex_Bypass(nn.Module):
    def __init__(self):
        super(SqueezeNet_Complex_Bypass, self).__init__()
        
        self.conv1 = ConvBlock(3, 96, kernel_size=7, stride=2, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire2 = Fire(96, 16)
        self.conv1x1_96 = ConvBlock(96, 128, kernel_size=1, stride=1, padding=0)
        
        self.fire3 = Fire(128, 16)
        self.conv1x1_128 = ConvBlock(128, 256, kernel_size=1, stride=1, padding=0)
        
        self.fire4 = Fire(128, 32)
        
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire5 = Fire(256, 32)
        self.conv1x1_256 = ConvBlock(256, 384, kernel_size=1, stride=1, padding=0)
        
        self.fire6 = Fire(256, 48)
        
        self.fire7 = Fire(384, 48)
        self.conv1x1_384 = ConvBlock(384, 512, kernel_size=1, stride=1, padding=0)
        
        self.fire8 = Fire(384, 64)
        
        self.maxpool8 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        
        self.fire9 = Fire(512, 64)
        
        self.dropout = nn.Dropout(0.5)
        
        self.conv10 = ConvBlock(512, 1000, kernel_size=1, stride=1, padding=0)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        
        pre_xc = self.conv1x1_96(x) # c: complex
        
        x = self.fire2(x)
        pre_xs = x
        
        x = self.fire3(pre_xc+x)
        pre_xc = self.conv1x1_128(x)
        
        x = self.fire4(pre_xs+x)
        
        x = self.maxpool4(pre_xc+x)
        pre_xs = x
        
        x = self.fire5(x)
        pre_xc = self.conv1x1_256(x)
        
        x = self.fire6(pre_xs+x)
        pre_xs = x
        
        x = self.fire7(pre_xc+x)
        pre_xc = self.conv1x1_384(x)
        
        x = self.fire8(pre_xs+x)
        
        x = self.maxpool8(pre_xc+x)
        pre_xs = x
        
        x = self.fire9(x)
        
        x = self.dropout(x)
        
        x = self.conv10(pre_xs+x)
        x = self.avgpool(x)
        
        x = x.view(x.size(0),-1)
        
        return x

In [7]:
if __name__ == '__main__':

    from torchsummary import summary
    model = SqueezeNet_Complex_Bypass()
    summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 111, 111]          14,112
              ReLU-2         [-1, 96, 111, 111]               0
         ConvBlock-3         [-1, 96, 111, 111]               0
         MaxPool2d-4           [-1, 96, 55, 55]               0
            Conv2d-5          [-1, 128, 55, 55]          12,288
              ReLU-6          [-1, 128, 55, 55]               0
         ConvBlock-7          [-1, 128, 55, 55]               0
            Conv2d-8           [-1, 16, 55, 55]           1,536
              ReLU-9           [-1, 16, 55, 55]               0
        ConvBlock-10           [-1, 16, 55, 55]               0
           Conv2d-11           [-1, 64, 55, 55]           1,024
             ReLU-12           [-1, 64, 55, 55]               0
        ConvBlock-13           [-1, 64, 55, 55]               0
           Conv2d-14           [-1, 64,